In [1]:
import numpy as np
import pandas as pd

In [2]:
df_files = pd.read_csv('/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/file_list_c.csv', sep = ',')
file_list = df_files['File_Name']

name_list = df_files['SN_Name']
path = '/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/'

In [3]:
df_TNS = pd.read_csv(path+'TNS_info_c.csv', sep = ',')

In [4]:
df_TNS['MJD']

0      57850.575995
1      57843.892998
2      57847.315000
3      57839.248993
4      57849.425995
           ...     
122    58089.167998
123    58100.640000
124    58103.070000
125    58102.052998
126    57767.320995
Name: MJD, Length: 127, dtype: float64

In [8]:
df_TNS

,Name,RA(deg),DEC(deg),MJD,SN_Redshift
0,2017ctt,232.508188,14.956263,57850.575995,0.090
1,2017cud,145.071375,-3.291850,57843.892998,0.068
2,2017cug,119.457833,-21.685839,57847.315000,0.024
3,2017cuh,83.717705,-20.046705,57839.248993,0.053
4,2017cxo,168.203083,31.920889,57849.425995,0.037
...,...,...,...,...,...
122,2017ivz,151.629292,-16.486881,58089.167998,0.079
123,2017iya,181.439833,10.398169,58100.640000,0.028
124,2017izu,32.199450,-44.666470,58103.070000,0.080
125,2017jdy,50.866417,-41.305569,58102.052998,0.070


In [5]:
with open('/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/ATLAS/ATLAS.LIST', 'w') as f:
    for i in range(0,len(file_list)):
        SN_name = str(df_files['SN_Name'][i])
        row = SN_name + '.dat' 
        f.write(row)
        f.write('\n')

In [6]:
for i in range(0,len(file_list)):
    
    
    SN_name = str(df_files['SN_Name'][i])
    RA = str(df_TNS['RA(deg)'][i])
    DEC = str(df_TNS['DEC(deg)'][i])
    MJD = str(df_TNS['MJD'][i])
    Redshift = str(df_TNS['SN_Redshift'][i])
    path1 = '/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/'
    print(path)
    df_ATLAS = pd.read_csv(path1 +file_list[i], sep = ',')


    MJD_ATLAS = df_ATLAS['MJD']
    mag_ATLAS = df_ATLAS['m']
    dmag_ATLAS = df_ATLAS['dm']
    filter_ATLAS = df_ATLAS['F']
    RA_deg = df_ATLAS['RA']
    DEC_deg = df_ATLAS['Dec']
    Sky_sig_ATLAS = df_ATLAS['Sky']

    flux = []
    fluxerr = []
    sky_flux = []
    for i in range(0,len(df_ATLAS),1):
        sky = Sky_sig_ATLAS[i]
        m = mag_ATLAS[i]
        dm = dmag_ATLAS[i]
        flux_calc = 10**(-0.4*(m - 27.5))
        #print(10**(-0.4*(m + dm - 27.5)),10**(-0.4*(m - 27.5)))
        fluxerr_calc = 10**(-0.4*(m - 27.5)) - 10**(-0.4*(m + dm - 27.5))
        sky_sig_calc = np.sqrt(10**(-0.4*(sky - 27.5)))
        #print(sky, sky_sig_calc)
    
        sky_flux.append(sky_sig_calc)
        flux.append(flux_calc)
        fluxerr.append(fluxerr_calc)
    
        if fluxerr_calc < 0:
            print("WARNING: Negative Flux")
            
    print(i)

    
    header_info = ['SURVEY: ATLAS', 'SNID: '+SN_name, 'IACU: ' +SN_name, 'FILTERS: co', 'TYPE: 1', 'RA: '+RA, 'DECL: '+DEC, 
               'PIXSIZE: 1.86','PEAKMJD: '+MJD, 'REDSHIFT_HELIO: '+Redshift+'  +- 0.000', 'HOSTGAL_OBJID: -9', 
               'HOSTGAL_PHOTOZ: -9','HOSTGAL_SPECZ: -9', 'HOSTGAL_SNSEP: -9', 'HOSTGAL_LOGMASS: -9', 'HOSTGAL_MAG: -9', 
               'HOSTGAL_SB_FLUXCAL: -9']
    rows = []
    for i in range(0,len(df_ATLAS),1):
        row = 'OBS: ' + str(round(MJD_ATLAS[i],6)) + ' ' + str(filter_ATLAS[i]) + ' ' + 'NA' + ' '+'{:0.5e}'.format(flux[i]) + ' ' + '{:0.4e}'.format(fluxerr[i]) + ' ' + str(round(sky_flux[i],1)) + ' '+ '1'+ ' '+ '0'
        rows.append(row)
    
    path2 = '/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT'
    with open(path2 +'/ATLAS/' + str(SN_name)+'.dat', 'w') as f:
        f.write('\n'.join(header_info))
        f.write('\n')
        f.write('# ================================')
        f.write('\n')
        f.write('# TERSE LIGHT CURVE OUTPUT')
        f.write('\n')
        f.write('#')
        f.write('\n')
        f.write('NOBS: '+str(len(df_ATLAS)))
        f.write('\n')
        f.write('NVAR: 8')
        f.write('\n')
        #f.write('VARLIST: MJD BAND FIELD FLUXCAL FLUXCALERR ZPFLUX PSF SKYSIG GAIN PHOTFLAG')
        f.write('VARLIST: MJD BAND FIELD FLUXCAL FLUXCALERR SKYSIG GAIN PHOTFLAG')
        f.write('\n')
        f.write('\n'.join(rows))
        f.write('\n')
        f.write('END_PHOTOMETRY:')

/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
0
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
1
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
2
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
3
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
4
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
5
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
6
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
7
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
8
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
9


95
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
96
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
97
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
98
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
99
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
100
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
101
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
102
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
103
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT/CLEAN_DATA/
104
/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTP

In [7]:
! ls ATLAS/*.dat | xargs -n1 basename > ATLAS/ATLAS.LIST

In [28]:
SN_name = str(df_TNS['Name'][0])
RA = str(df_TNS['RA(deg)'][0])
DEC = str(df_TNS['DEC(deg)'][0])
MJD = str(df_TNS['MJD'][0])
Redshift = str(df_TNS['SN_Redshift'][0])

In [29]:
header_info = ['SURVEY: ATLAS', 'SNID: '+SN_name, 'IACU: ' +SN_name, 'FILTERS: co', 'TYPE: 1', 'RA: '+RA, 'DECL: '+DEC, 
               'PIXSIZE: 1.86','PEAKMJD: '+MJD, 'REDSHIFT_HELIO: '+Redshift+'  +- 0.000', 'HOSTGAL_OBJID: -9', 
               'HOSTGAL_PHOTOZ: -9','HOSTGAL_SPECZ: -9', 'HOSTGAL_SNSEP: -9', 'HOSTGAL_LOGMASS: -9', 'HOSTGAL_MAG: -9', 
               'HOSTGAL_SB_FLUXCAL: -9']

In [15]:
'{:0.5e}'.format(flux[0])

'2.07778e-03'

In [26]:
rows = []
for i in range(0,len(df_ATLAS),1):
    #row = 'OBS: ' + str(round(MJD_ATLAS[i],6)) + ' ' + str(filter_ATLAS[i]) + ' ' + 'NA' + ' '+'{:0.5e}'.format(flux[i]) + ' ' + '{:0.4e}'.format(fluxerr[i]) + ' '+ 'NA'+ ' '+ 'NA' ' ' + '27.5' + ' '+ '1'+ ' '+ '0'
    row = 'OBS: ' + str(round(MJD_ATLAS[i],6)) + ' ' + str(filter_ATLAS[i]) + ' ' + 'NA' + ' '+'{:0.5e}'.format(flux[i]) + ' ' + '{:0.4e}'.format(fluxerr[i]) + ' ' + str(round(sky_flux[i],1)) + ' '+ '1'+ ' '+ '0'
    rows.append(row)
                        

In [28]:
path = '/project2/rkessler/SURVEYS/LSST/USERS/lmercadante/ATLAS_DATA_PROJECT/ATLAS_API_OUTPUT'
with open(path +'/ATLAS/' + str(SN_name)+'.dat', 'w') as f:
    f.write('\n'.join(header_info))
    f.write('\n')
    f.write('# ================================')
    f.write('\n')
    f.write('# TERSE LIGHT CURVE OUTPUT')
    f.write('\n')
    f.write('#')
    f.write('\n')
    f.write('NOBS: '+str(len(df_ATLAS)))
    f.write('\n')
    f.write('NVAR: 8')
    f.write('\n')
    #f.write('VARLIST: MJD BAND FIELD FLUXCAL FLUXCALERR ZPFLUX PSF SKYSIG GAIN PHOTFLAG')
    f.write('VARLIST: MJD BAND FIELD FLUXCAL FLUXCALERR SKYSIG GAIN PHOTFLAG')
    f.write('\n')
    f.write('\n'.join(rows))
    f.write('\n')
    f.write('END_PHOTOMETRY:')